In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import time

import utils

In [2]:
# Hyperparameter initialization and train_set size and 
# test set size
learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000

# Download the dataset and pre process it for training and testing
mnist_folder = 'data/mnist'
utils.download_mnist(mnist_folder)
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists


In [3]:
# Create the datasets and the iterators
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000)
train_data = train_data.batch(batch_size) # Create batches of 128 examples 

# Training dataset
test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

In [4]:
# Create one iterator and initialize it with different datasets

iterator = tf.data.Iterator.from_structure(train_data.output_types,
                                          train_data.output_shapes)

img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

In [5]:
# Create the weights and bias
w = tf.get_variable(initializer=tf.random_normal_initializer(0, 0.01), shape=(784, 10), name='weights')
b = tf.get_variable(initializer=tf.zeros_initializer(), shape=(1, 10), name='bias')

In [6]:
logits = tf.matmul(img, w) + b

entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label, name='entropy')
loss = tf.reduce_mean(entropy, name='loss')

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [7]:
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('./graphs/logreg', tf.get_default_graph())

In [10]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    
    for i in range(n_epochs):
        sess.run(train_init)
        total_loss = 0
        n_batches = 0
    
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        print('Average loss epoch {0} {1}'.format(i, total_loss/n_batches))
        
    print('Total Time: {0} seconds'.format(time.time() - start_time))
    
    sess.run(test_init)
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass
    
    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0 0.3630769000843514
Average loss epoch 1 0.29272632980069446
Average loss epoch 2 0.2810420200228691
Average loss epoch 3 0.2769210001583709
Average loss epoch 4 0.27500324777738994
Average loss epoch 5 0.27007326097335926
Average loss epoch 6 0.2685691674088323
Average loss epoch 7 0.2663046835640142
Average loss epoch 8 0.26622128384404403
Average loss epoch 9 0.2659454184042853
Average loss epoch 10 0.2627765771088212
Average loss epoch 11 0.26139979771403377
Average loss epoch 12 0.26058941441912986
Average loss epoch 13 0.26055967168405997
Average loss epoch 14 0.2585872258904368
Average loss epoch 15 0.2593832071437392
Average loss epoch 16 0.25791870889275575
Average loss epoch 17 0.2540261521935463
Average loss epoch 18 0.25339071100880933
Average loss epoch 19 0.2543319433407728
Average loss epoch 20 0.2535630573366964
Average loss epoch 21 0.2586546514616456
Average loss epoch 22 0.2530089950145677
Average loss epoch 23 0.25335650197988335
Average loss epo